In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import urllib
import time
import feedparser
import pickle
import os, operator, warnings
import logging

logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
%matplotlib inline

import gensim
from gensim import corpora, models, similarities
from gensim.matutils import Sparse2Corpus
import gensim.parsing.preprocessing as genpre
from time import time
from nltk.stem.wordnet import WordNetLemmatizer

import numpy as np
import spacy

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
import pyLDAvis.gensim

INFO : 'pattern' package not found; tag filters are not available for English


In [19]:
abstracts = pd.read_csv('./new_hope_data/arxiv_csML.csv')['summary']

In [2]:
# Gonna import anew. Do my own de-LATEX-ing, then remove numerics
# punctuations and stops
lmtzr= WordNetLemmatizer()
def prep_text(text):
     #this removes LATEX formatting, citations, splits hyphens into two words (potential to )
    myreg=r'\\[\w]+[\{| ]|\$[^\$]+\$|\(.+\, *\d{2,4}\w*\)|\S*\/\/\S*|[\\.,\/#!$%\^&\*;:{}=_`\'\"~()><\|]|\[.+\]|\d+|\b\w{1,2}\b'
    parsed_data = text.replace('-', ' ')
    parsed_data = re.sub(myreg, '', parsed_data)
    parsed_data = [lmtzr.lemmatize(w) for w in parsed_data.lower().split() if w not in genpre.STOPWORDS]
    return parsed_data
    # Use gensim
#     genpre.
    
#     tokens = [str(token).lower() for token in parsed_data if len(token) > 3]
#     tokens = [token for token in tokens if token not in STOPWORDS]
#     tokens = [get_lemma(token) for token in tokens]
#     return tokens

In [104]:
# HELPER FUNCS TO TEST OUT MY PREPROC FUNC

#  test with about 200 abstracts
# start= 99
# end = 299
# tester = [prep_text(ab) for ab in abstracts[start:end]]

# print(list(zip(range(start,end),tester)))

# ab = 157
# print(abstracts[ab], '\n')
# print(prep_text(abstracts[ab]))

# thegoods = [prep_text(ab) for ab in abstracts]
# with open('./the_data_strikes_back/the_goods', 'wb') as fp:
#     pickle.dump(thegoods, fp)

In [12]:
with open('./the_data_strikes_back/the_goods', 'rb') as fp:
    corpus = pickle.load(fp)

In [4]:
dictionary =corpora.Dictionary(corpus)

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : adding document #10000 to Dictionary(20222 unique tokens: ['algorithm', 'applied', 'assuming', 'automaton', 'cellular']...)
INFO : built Dictionary(29474 unique tokens: ['algorithm', 'applied', 'assuming', 'automaton', 'cellular']...) from 18961 documents (total 1689563 corpus positions)


In [13]:
bigrams = gensim.models.Phrases(corpus)

INFO : collecting all words and their counts
INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO : PROGRESS: at sentence #10000, processed 870367 words and 446264 word types
INFO : collected 767458 word types from a corpus of 1689563 words (unigram + bigrams) and 18961 sentences
INFO : using 767458 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


In [14]:
texts = [bigrams[line] for line in corpus]

/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [15]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
# Then filter, etc. coool!

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : adding document #10000 to Dictionary(26068 unique tokens: ['algorithm', 'applied', 'assuming', 'automaton', 'cellular']...)
INFO : built Dictionary(35661 unique tokens: ['algorithm', 'applied', 'assuming', 'automaton', 'cellular']...) from 18961 documents (total 1479562 corpus positions)


In [22]:
lsi = LsiModel(corpus = corpus, num_topics = 10, id2word=dictionary)
lsi.show_topics(num_topics=5)

INFO : using serial LSI version on this node
INFO : updating model with new documents
INFO : preparing a new chunk of documents
INFO : using 100 extra samples and 2 power iterations
INFO : 1st phase: constructing (35661, 110) action matrix
INFO : orthonormalizing (35661, 110) action matrix
INFO : 2nd phase: running dense svd on (110, 18961) matrix
INFO : computing the final decomposition
INFO : keeping 10 factors (discarding 51.977% of energy spectrum)
INFO : processed documents up to #18961
INFO : topic #0(472.020): 0.471*"model" + 0.329*"method" + 0.306*"data" + 0.285*"algorithm" + 0.185*"problem" + 0.176*"based" + 0.171*"learning" + 0.167*"approach" + 0.153*"network" + 0.105*"proposed"
INFO : topic #1(252.064): 0.797*"model" + -0.413*"algorithm" + -0.266*"method" + -0.199*"problem" + -0.097*"learning" + -0.063*"function" + -0.062*"based" + -0.053*"proposed" + -0.052*"matrix" + -0.048*"kernel"
INFO : topic #2(210.544): -0.696*"algorithm" + 0.581*"method" + 0.242*"data" + -0.195*"mode

[(0,
  '0.471*"model" + 0.329*"method" + 0.306*"data" + 0.285*"algorithm" + 0.185*"problem" + 0.176*"based" + 0.171*"learning" + 0.167*"approach" + 0.153*"network" + 0.105*"proposed"'),
 (1,
  '0.797*"model" + -0.413*"algorithm" + -0.266*"method" + -0.199*"problem" + -0.097*"learning" + -0.063*"function" + -0.062*"based" + -0.053*"proposed" + -0.052*"matrix" + -0.048*"kernel"'),
 (2,
  '-0.696*"algorithm" + 0.581*"method" + 0.242*"data" + -0.195*"model" + 0.129*"network" + -0.102*"problem" + 0.074*"feature" + -0.061*"learning" + 0.060*"approach" + 0.043*"kernel"'),
 (3,
  '0.785*"data" + -0.518*"method" + -0.185*"model" + -0.115*"problem" + 0.099*"network" + 0.058*"training" + 0.056*"approach" + 0.054*"learning" + -0.050*"function" + 0.047*"analysis"'),
 (4,
  '-0.806*"network" + 0.322*"data" + -0.211*"learning" + 0.207*"method" + -0.123*"task" + 0.122*"model" + -0.107*"graph" + -0.102*"training" + -0.094*"node" + -0.080*"neural_network"')]

In [23]:
hdpmodel = HdpModel(corpus=corpus, id2word=dictionary)
hdpmodel.show_topics()

INFO : (0, '0.015*model + 0.013*method + 0.013*algorithm + 0.011*data + 0.009*problem + 0.007*based + 0.007*approach + 0.007*learning + 0.005*network + 0.005*result')
INFO : (1, '0.015*model + 0.012*algorithm + 0.012*method + 0.011*data + 0.008*problem + 0.007*based + 0.006*learning + 0.006*approach + 0.005*network + 0.004*result')
INFO : (2, '0.013*model + 0.012*algorithm + 0.011*method + 0.009*data + 0.008*problem + 0.006*approach + 0.005*based + 0.005*learning + 0.004*result + 0.004*network')
INFO : (3, '0.013*algorithm + 0.012*model + 0.012*method + 0.009*data + 0.008*problem + 0.006*based + 0.006*approach + 0.005*learning + 0.003*result + 0.003*network')
INFO : (4, '0.011*model + 0.010*algorithm + 0.010*method + 0.008*data + 0.006*problem + 0.005*approach + 0.005*based + 0.005*learning + 0.003*function + 0.003*result')
INFO : (5, '0.008*model + 0.007*data + 0.007*algorithm + 0.006*method + 0.005*problem + 0.004*based + 0.003*learning + 0.003*approach + 0.003*feature + 0.002*result

INFO : (13, '0.002*algorithm + 0.001*learning + 0.001*set + 0.001*method + 0.001*model + 0.001*data + 0.001*problem + 0.001*based + 0.001*gmic + 0.001*approach')
INFO : (14, '0.001*method + 0.001*model + 0.001*process + 0.001*data + 0.001*information + 0.001*algorithm + 0.001*function + 0.001*group + 0.001*problem + 0.001*factor')
INFO : (15, '0.001*method + 0.001*data + 0.001*graph + 0.001*algorithm + 0.001*bootstrap + 0.001*model + 0.001*application + 0.001*statistical + 0.001*based + 0.001*number')
INFO : (16, '0.002*method + 0.002*model + 0.001*distribution + 0.001*problem + 0.001*algorithm + 0.001*data + 0.001*variable + 0.001*based + 0.001*case + 0.001*mloc')
INFO : (17, '0.001*model + 0.001*algorithm + 0.001*node + 0.001*data + 0.001*tree + 0.001*regression + 0.001*sparse + 0.001*estimation + 0.001*polynomial + 0.001*solution')
INFO : (18, '0.002*model + 0.001*data + 0.001*dimensionality_reduction + 0.001*dynamic + 0.001*time + 0.001*based + 0.001*sparse + 0.001*information + 0.

[(0,
  '0.016*model + 0.013*method + 0.011*algorithm + 0.011*data + 0.008*problem + 0.007*learning + 0.007*based + 0.007*approach + 0.006*network + 0.005*result + 0.004*proposed + 0.004*task + 0.004*performance + 0.004*training + 0.004*function + 0.004*feature + 0.003*new + 0.003*framework + 0.003*propose + 0.003*distribution'),
 (1,
  '0.016*model + 0.012*method + 0.011*data + 0.011*algorithm + 0.007*problem + 0.007*based + 0.007*approach + 0.007*network + 0.007*learning + 0.004*result + 0.004*proposed + 0.004*task + 0.003*performance + 0.003*feature + 0.003*training + 0.003*function + 0.003*new + 0.003*framework + 0.003*distribution + 0.003*time'),
 (2,
  '0.015*model + 0.012*algorithm + 0.011*method + 0.010*data + 0.008*problem + 0.006*based + 0.006*approach + 0.006*learning + 0.005*network + 0.004*result + 0.004*function + 0.004*proposed + 0.003*performance + 0.003*feature + 0.003*graph + 0.003*task + 0.003*distribution + 0.003*analysis + 0.003*propose + 0.003*parameter'),
 (3,
  '

In [25]:
ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
ldamodel.show_topics()

INFO : using symmetric alpha at 0.1
INFO : using symmetric eta at 0.1
INFO : using serial LDA version on this node
INFO : running online (single-pass) LDA training, 10 topics, 1 passes over the supplied corpus of 18961 documents, updating model once every 2000 documents, evaluating perplexity every 18961 documents, iterating 50x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #2000/18961
INFO : merging changes from 2000 documents into a model of 18961 documents
INFO : topic #3 (0.100): 0.025*"model" + 0.018*"method" + 0.013*"algorithm" + 0.010*"learning" + 0.009*"based" + 0.008*"data" + 0.008*"problem" + 0.006*"approach" + 0.005*"result" + 0.005*"distribution"
INFO : topic #8 (0.100): 0.023*"model" + 0.015*"algorithm" + 0.014*"method" + 0.010*"data" + 0.010*"problem" + 0.008*"based" + 0.008*"approach" + 0.008*"graph" + 0.007*"result" + 0.006*"learning"
INFO : topic #4 (0.100): 0.015*"algorithm" + 0.012*"model" + 0.009*"data" + 0.009*"network" + 0.009*"stru

INFO : topic #6 (0.100): 0.020*"kernel" + 0.016*"method" + 0.012*"data" + 0.011*"feature" + 0.009*"model" + 0.009*"class" + 0.008*"algorithm" + 0.007*"based" + 0.006*"approach" + 0.006*"new"
INFO : topic #7 (0.100): 0.016*"data" + 0.012*"method" + 0.010*"model" + 0.008*"feature" + 0.007*"approach" + 0.007*"analysis" + 0.006*"based" + 0.006*"signal" + 0.006*"proposed" + 0.005*"network"
INFO : topic diff=0.651928, rho=0.377964
INFO : PROGRESS: pass 0, at document #16000/18961
INFO : merging changes from 2000 documents into a model of 18961 documents
INFO : topic #8 (0.100): 0.027*"model" + 0.014*"algorithm" + 0.012*"method" + 0.010*"problem" + 0.008*"approach" + 0.008*"data" + 0.008*"based" + 0.008*"estimation" + 0.007*"tensor" + 0.006*"result"
INFO : topic #9 (0.100): 0.022*"data" + 0.017*"algorithm" + 0.014*"matrix" + 0.014*"problem" + 0.011*"method" + 0.007*"approach" + 0.007*"learning" + 0.007*"point" + 0.006*"proposed" + 0.005*"based"
INFO : topic #7 (0.100): 0.017*"data" + 0.012*"m

/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(El

/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(El

/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(El

...

/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(El

/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(El

/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(El

INFO : PROGRESS: pass 0, at document #18961/18961
INFO : merging changes from 961 documents into a model of 18961 documents
INFO : topic #9 (0.100): 0.022*"data" + 0.016*"algorithm" + 0.014*"matrix" + 0.014*"problem" + 0.012*"method" + 0.007*"approach" + 0.007*"learning" + 0.006*"point" + 0.006*"proposed" + 0.005*"based"
INFO : topic #0 (0.100): 0.015*"algorithm" + 0.012*"problem" + 0.010*"learning" + 0.010*"result" + 0.009*"function" + 0.009*"bound" + 0.007*"setting" + 0.007*"rate" + 0.007*"distribution" + 0.006*"study"
INFO : topic #7 (0.100): 0.019*"data" + 0.013*"method" + 0.009*"model" + 0.008*"feature" + 0.007*"approach" + 0.007*"based" + 0.006*"proposed" + 0.006*"analysis" + 0.006*"signal" + 0.005*"study"
INFO : topic #6 (0.100): 0.017*"method" + 0.017*"kernel" + 0.012*"data" + 0.011*"feature" + 0.010*"class" + 0.008*"model" + 0.007*"approach" + 0.007*"based" + 0.006*"algorithm" + 0.006*"problem"
INFO : topic #4 (0.100): 0.059*"network" + 0.025*"graph" + 0.013*"deep_neural" + 0.

[(0,
  '0.015*"algorithm" + 0.012*"problem" + 0.010*"learning" + 0.010*"result" + 0.009*"function" + 0.009*"bound" + 0.007*"setting" + 0.007*"rate" + 0.007*"distribution" + 0.006*"study"'),
 (1,
  '0.027*"algorithm" + 0.026*"method" + 0.013*"problem" + 0.010*"function" + 0.009*"optimization" + 0.008*"model" + 0.008*"gradient" + 0.008*"based" + 0.006*"learning" + 0.006*"approach"'),
 (2,
  '0.017*"task" + 0.016*"model" + 0.013*"learning" + 0.011*"training" + 0.009*"approach" + 0.008*"image" + 0.008*"neural_network" + 0.008*"data" + 0.006*"method" + 0.006*"representation"'),
 (3,
  '0.044*"model" + 0.018*"method" + 0.012*"data" + 0.011*"distribution" + 0.009*"learning" + 0.008*"training" + 0.008*"sample" + 0.008*"based" + 0.007*"approach" + 0.006*"network"'),
 (4,
  '0.059*"network" + 0.025*"graph" + 0.013*"deep_neural" + 0.012*"node" + 0.011*"model" + 0.010*"structure" + 0.010*"layer" + 0.009*"deep" + 0.006*"data" + 0.006*"method"'),
 (5,
  '0.027*"model" + 0.018*"data" + 0.011*"based" 

In [26]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/home/omar/miniconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.080716 -0.014694       1        1  15.338161
2     -0.113737 -0.018584       2        1  14.841501
7     -0.080170 -0.050509       3        1  10.166489
0      0.161622  0.059110       4        1   9.671205
8      0.080147 -0.003544       5        1   9.069558
3     -0.039231  0.006205       6        1   8.720962
9      0.081365 -0.027602       7        1   8.574484
5     -0.085484 -0.075224       8        1   8.343725
4     -0.103338  0.168372       9        1   7.850307
6      0.018112 -0.043530      10        1   7.423608, topic_info=     Category          Freq              Term         Total  loglift  logprob
term                                                                         
439   Default   9542.000000           network   9542.000000  30.0000  30.0000
82    Default  22328.000000             model  22328.000000  29.0000  29.0000
346   Default   3946.000000             graph   3946.000000  28.0000  28.0000
190   Default   2446.000000            kernel   2446.000000  27.0000  27.0000
7347  Default   1774.000000       deep_neural   1774.000000  26.0000  26.0000
0     Default  15629.000000         algorithm  15629.000000  25.0000  25.0000
353   Default   2599.000000            matrix   2599.000000  24.0000  24.0000
217   Default  15904.000000              data  15904.000000  23.0000  23.0000
118   Default   5094.000000              task   5094.000000  22.0000  22.0000
1179  Default   1471.000000              node   1471.000000  21.0000  21.0000
193   Default  18702.000000            method  18702.000000  20.0000  20.0000
5097  Default   1549.000000             layer   1549.000000  19.0000  19.0000
342   Default   4281.000000      distribution   4281.000000  18.0000  18.0000
983   Default   1926.000000              deep   1926.000000  17.0000  17.0000
3706  Default   2230.000000          gradient   2230.000000  16.0000  16.0000
644   Default   5493.000000          training   5493.000000  15.0000  15.0000
63    Default   5043.000000           feature   5043.000000  14.0000  14.0000
230   Default   4983.000000          function   4983.000000  13.0000  13.0000
101   Default  10926.000000           problem  10926.000000  12.0000  12.0000
690   Default   1710.000000             bound   1710.000000  11.0000  11.0000
88    Default   2592.000000      optimization   2592.000000  10.0000  10.0000
1152  Default   2439.000000             image   2439.000000   9.0000   9.0000
1951  Default   2126.000000            policy   2126.000000   8.0000   8.0000
800   Default   1625.000000         estimator   1625.000000   7.0000   7.0000
756   Default   3300.000000             class   3300.000000   6.0000   6.0000
451   Default   3117.000000         structure   3117.000000   5.0000   5.0000
2121  Default   3213.000000    neural_network   3213.000000   4.0000   4.0000
623   Default   4104.000000            sample   4104.000000   3.0000   3.0000
5357  Default    779.000000            tensor    779.000000   2.0000   2.0000
3498  Default   1650.000000      architecture   1650.000000   1.0000   1.0000
...       ...           ...               ...           ...      ...      ...
779   Topic10    529.273743             prior   1644.659668   1.4667  -5.3353
3012  Topic10    159.331512              base    279.486145   2.0385  -6.5358
9209  Topic10    138.965836        deployment    225.810959   2.1150  -6.6726
1248  Topic10    244.791794         posterior    565.953186   1.7624  -6.1064
852   Topic10    547.126648              test   2234.237061   1.1935  -5.3021
193   Topic10   1899.855713            method  18702.837891   0.3136  -4.0573
818   Topic10    427.653778          sampling   1518.647583   1.3332  -5.5485
249   Topic10    466.701233        regression   1947.162109   1.1721  -5.4611
753   Topic10    396.579224          bayesian   1502.367554   1.2686  -5.6239
217   Topic10   1322.695923      

# OK, my goal is clear#

I want to make a paper reccomender system based on Topic modeling. You input the abstract, it processes it and puts it thru the model, the model colors the words in the abstract based on topic, and then returns the most similar abstracts to that based on topic similarity. Sweet idea!